In [ ]:
## EU-DV-2 Threshold definition  – time-series
# The interface allow the expert user to define a threshold and select a 
# pollutant, a time window and a province and visualize a time series showing which days the 
# threshold has been exceeded and a percentage (exceeded days/total days).

# - button select a pollutant
# - button select threshold
# - button select start day
# - button select end day
# - button select start mounth
# - button select end mounth
# - button select start year
# - button select end year
# - button select a province 


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import folium
from folium import Map, Marker
from IPython.display import IFrame
import branca
import requests
import pandas as pd
import geopandas as gpd
from shapely import wkt
import ipywidgets as widgets
import matplotlib.pyplot as plt
import folium
from shapely.geometry import mapping
import leafmap as leafmap
from shapely.geometry import shape

# Output dinamici
output1 = widgets.Output()
output2 = widgets.Output()
output_map = widgets.Output()

# Dropdown inquinanti
dropdown = widgets.Dropdown(
    options=[
        "Ammoniaca", "Arsenico", "Benzo(a)pirene", "Benzene", "Biossido di Azoto",
        "Biossido di Zolfo", "BlackCarbon", "Cadmio", "Monossido di Azoto",
        "Monossido di Carbonio", "Nikel", "Ossidi di Azoto", "Ozono", "PM10",
        "PM10 (SM2005)", "Particelle sospese PM2.5", "Particolato Totale Sospeso",
        "Piombo"
    ],
    description='Inquinante:'
)

# Funzione per generare mappa Folium centrata sulla Lombardia
def show_lombardia_map():
    with output_map:
        clear_output()
        # Centro approssimativo Lombardia
        m = folium.Map(location=[45.5, 9.5], zoom_start=8)
        display(m)

# Funzione quando cambia dropdown: mostra slider e collega evento
def on_dropdown_change1(change):
    if change['type'] == 'change' and change['name'] == 'value':
        with output1:
            clear_output()
            slider = widgets.IntSlider(
                value=0,
                min=0,
                max=80,
                step=1,
                description=f'Soglia ({change["new"]}):',
                continuous_update=False
            )
            display(slider)

            def on_slider_change2(slider_change):
                if slider_change['type'] == 'change' and slider_change['name'] == 'value':
                    with output2:
                        clear_output()

                        # Date picker iniziale
                        start_date = widgets.DatePicker(description='Inizio:')
                        end_date = widgets.DatePicker(description='Fine:')

                        # Bottone di conferma date
                        confirm_button = widgets.Button(description='Conferma intervallo')

                        # Funzione al click del bottone
                        def on_confirm_clicked(b):
                            if start_date.value and end_date.value:
                                show_lombardia_map()

                        confirm_button.on_click(on_confirm_clicked)

                        display(widgets.VBox([start_date, end_date, confirm_button]))

                    def get_province_shape():
                        t=requests.get(url="http://127.0.0.1:5000/api/province_shape")
                        data = read_response(t)
                        gdf = gpd.GeoDataFrame(data)
                        gdf['geometry_province'] = gdf['geometry_province'].apply(wkt.loads)
                        gdf.set_geometry('geometry_province', inplace=True)
                        return gdf
                    prov=get_province_shape()

            slider.observe(on_slider_change2, names='value')

dropdown.observe(on_dropdown_change1, names='value')

# Mostra tutto
display(dropdown)
display(output1)
display(output2)
display(output_map)



Dropdown(description='Inquinante:', options=('Ammoniaca', 'Arsenico', 'Benzo(a)pirene', 'Benzene', 'Biossido d…

Output()

Output()

Output()

NameError: name 'requests' is not defined

In [ ]:
# EU-DV-2 

data = {"var_pollutant": "Benzene", "var_start_date" :"2024-12-19 18:00:00", "var_end_date" :"2025-01-01 00:00:00","var_province":"MILANO"} #Right now data is a dictionary
t=requests.post(url="http://127.0.0.1:5000/api/EU_DV_2", json=data) 

try:
    data = t.json() #This will convert the response to a json object
    print("The query returned", len(data), "rows")
    print(type(data))
except requests.exceptions.JSONDecodeError:
    print("Risposta non valida JSON!")
    print("Contenuto della risposta:", t.text)
    data = None

df = pd.DataFrame(data)
print(df)